In [1]:
import tensorflow as tf
from keras.models import Sequential,Model
from keras.layers import Input,Convolution2D,Dense,LeakyReLU,Reshape,Flatten,MaxPooling2D,LSTM,Dropout,TimeDistributed
import numpy as np
from frames import FrameData
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
fd = FrameData(".",dim = (320,320))
width = 320
height = 320
channels = 3

In [3]:
width
timesteps = 10

In [19]:
model = Sequential()
x_input = Input(shape=[width,height,channels])
x_output = Dense(1024)(Flatten()(Convolution2D(filters=64, kernel_size=4, activation='relu')(x_input)))
base_model = Model(x_input, x_output)
model.add(TimeDistributed(base_model, input_shape=base_model.input_shape))
#model.add(TimeDistributed(Flatten(input_shape=base_model.input_shape[1:])))


model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_13 (TimeDis (None, None, 1024)        6585651264
Total params: 6,585,651,264
Trainable params: 6,585,651,264
Non-trainable params: 0
_________________________________________________________________


In [32]:

inp = Input(shape=[width,height,channels])
enc = Convolution2D(512,kernel_size= 5,strides = [2,2],padding = "same")(inp)
enc = MaxPooling2D(pool_size = 2,strides = [2,2],padding = "same")(enc)
enc = LeakyReLU(0.6)(enc)
enc = Convolution2D(128,kernel_size = 5,strides = [2,2],padding = "same")(enc)
enc = MaxPooling2D(pool_size = 2,strides = [2,2],padding = "same")(enc)
enc = LeakyReLU(0.6)(enc)
enc = Convolution2D(32,kernel_size = 5,strides = [2,2],padding = "same")(enc)
enc = MaxPooling2D(pool_size = 2,strides = [2,2],padding = "same")(enc)
enc = LeakyReLU(0.6)(enc)
enc = Flatten()(enc)
enc = Dense(1024,activation = "sigmoid")(enc)
encoder = Model(inputs = [inp],outputs = [enc])
model = Sequential()
model.add(TimeDistributed(encoder,input_shape = encoder.input_shape))
model.add(LSTM(100))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_19 (TimeDis (None, None, 1024)        2600096   
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               450000    
Total params: 3,050,096
Trainable params: 3,050,096
Non-trainable params: 0
_________________________________________________________________


In [34]:
help(TimeDistributed)

Help on class TimeDistributed in module keras.layers.wrappers:

class TimeDistributed(Wrapper)
 |  This wrapper applies a layer to every temporal slice of an input.
 |  
 |  The input should be at least 3D, and the dimension of index one
 |  will be considered to be the temporal dimension.
 |  
 |  Consider a batch of 32 samples,
 |  where each sample is a sequence of 10 vectors of 16 dimensions.
 |  The batch input shape of the layer is then `(32, 10, 16)`,
 |  and the `input_shape`, not including the samples dimension, is `(10, 16)`.
 |  
 |  You can then use `TimeDistributed` to apply a `Dense` layer
 |  to each of the 10 timesteps, independently:
 |  
 |  ```python
 |      # as the first layer in a model
 |      model = Sequential()
 |      model.add(TimeDistributed(Dense(8), input_shape=(10, 16)))
 |      # now model.output_shape == (None, 10, 8)
 |  ```
 |  
 |  The output will then have shape `(32, 10, 8)`.
 |  
 |  In subsequent layers, there is no need for the `input_shape`:
 

In [13]:
model.compile(optimizer  ="adam" , loss = "mean_squared_error")

In [14]:
model.summary()

ValueError: This model has never been called, this its weights have not yet been created, so no summary can be displayed. Build the model first (e.g. by calling it on some test data).

In [33]:
timesteps = 10
sequence_pred = Sequential([
                           ])
sequence_pred.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 10, 30)            126600    
_________________________________________________________________
dropout_4 (Dropout)          (None, 10, 30)            0         
Total params: 126,600
Trainable params: 126,600
Non-trainable params: 0
_________________________________________________________________


In [154]:
inp = Input(shape = [320,320,3])
l = encoder(inp)
l = decoder(l)
autoenc = Model(inputs = [inp],outputs = [l])

In [155]:
autoenc.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 320, 320, 3)       0         
_________________________________________________________________
sequential_25 (Sequential)   (None, 64)                3110350048
_________________________________________________________________
sequential_26 (Sequential)   (None, 320, 320, 3)       3113385603
Total params: 6,223,735,651
Trainable params: 6,223,735,651
Non-trainable params: 0
_________________________________________________________________
